In [1]:
import sys

In [2]:
sys.path.append('.')

In [3]:
import tools_graph
import tools_data

In [4]:
experiment_set = "data/freqV3"
experiments = tools_data.getExperimentsInSet(experiment_set)

In [5]:
experiments

['5_8_112500_1_0_0_0_20210914_1104',
 '5_6_180000_1_0_0_0_20210914_1132',
 '5_10_90000_1_0_0_0_20210914_1035',
 '5_100_9000_1_0_0_0_20210914_1007',
 '5_1000_1000_1_0_0_0_20210914_0938']

In [6]:
for experiment in experiments:
    print(experiment)
    params = experiment.split("_")
    params = [int(p) for p in params]

    consumer = tools_data.readConsumer(experiment_set, experiment)
    device_aggregate = tools_data.readAggregateDevices(experiment_set, experiment)
    tools_graph.graphType1(device_aggregate, consumer, "{} devices, one message every {} ms, {} messages per second".format(params[0], int(1000/params[1]), int(params[0]*(1000/params[1]))))
    tools_graph.graphType2(consumer, "{} devices, one message every {} ms, {} messages per second".format(params[0], int(1000/params[1]), int(params[0]*(1000/params[1]))))

5_8_112500_1_0_0_0_20210914_1104


ValueError: Unable to parse string "None" at position 0

In [ ]:
tools_graph.graphType3(experiment_set, 1, "Messages per second", experiment_set, lambda params: int(params[0]) * (1000/int(params[1])))

In [ ]:
tools_graph.graphType4(experiment_set, 1, "Messages per second",  experiment_set, lambda params: int(params[0]) * (1000/int(params[1])))